# Lab 2: LangGraph Components


## Setting Up the Environment

In [ ]:
from dotenv import load_dotenv
import json
import os
import re
import sys
import warnings

import boto3
from botocore.config import Config

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)

# Create a bedrock client to check available models
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)


## LangGraph as a State Machine

For solution architects familiar with system design, LangGraph can be thought of as a state machine for language models. Just as a state machine in software engineering defines a set of states and transitions between them, LangGraph allows us to define the states of our conversation (represented by nodes) and the transitions between them (represented by edges).

**Analogy**: Think of LangGraph as a traffic control system for a smart city. Each intersection (node) represents a decision point, and the roads between them (edges) represent possible paths. The traffic lights (conditional edges) determine which path to take based on current conditions. In our case, the "traffic" is the flow of information and decisions in our AI agent.


In [ ]:
import operator
from typing import Annotated, TypedDict

from langchain_aws import ChatBedrockConverse
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langgraph.graph import END, StateGraph

In [ ]:
tool = TavilySearchResults(max_results=4)  # increased number of results
print(type(tool))
print(tool.name)

> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).


## The Agent State Concept

The AgentState class is crucial for maintaining context throughout the conversation. For data scientists, this can be compared to maintaining state in a recurrent neural network.

**Analogy**: Think of the AgentState as a sophisticated notepad. As you brainstorm ideas (process queries), you jot down key points (messages). This notepad doesn't just record; it has a special property where new notes (messages) are seamlessly integrated with existing ones, maintaining a coherent flow of thought.
At the same time, you can always go back in time and rewrite some parts of it - this is what we call "time-travel".


In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name.

```python
if not t["name"] in self.tools:  # check for bad tool name from LLM
    print("\n ....bad tool name....")
    result = "bad tool name, retry"  # instruct LLM to retry if bad

```


In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_bedrock)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_bedrock(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t["name"] in self.tools:  # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

An often overlooked feature is the `??` to inspect the code of a function or object in python.

Lets inspect the `bind_tools` method on the `ChatBedrockConverse` class.
Can you spot if our tavily tool will be supported, and if there are any restrictions?

If you are unsure, how would you check?


In [ ]:
??ChatBedrockConverse.bind_tools

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence).\
Whenever you can, try to call multiple tools at once, to bring down inference time!\
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
)

abot = Agent(model, [tool], system=prompt)

In [ ]:
# make sure to install pygraphviz if you haven't done so already using 'conda install --channel conda-forge pygraphviz'
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
for message in result["messages"]:
    print(f"{message}\n")

In [ ]:
result["messages"][-1].content

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result["messages"][-1].content

## 4. Parallel vs. Sequential Tool Calling

The ability of the agent to make both parallel and sequential tool calls is a powerful feature that solution architects should pay attention to.

**Deep Dive**:

- Parallel tool calling is like a multithreaded application, where multiple independent tasks can be executed simultaneously. This is efficient for queries that require multiple pieces of independent information.
- Sequential tool calling is more like a pipeline, where the output of one operation becomes the input of the next. This is necessary for multi-step reasoning tasks.

**Analogy**: Imagine a research team working on a complex project. Parallel tool calling is like assigning different team members to research different aspects simultaneously. Sequential tool calling is like a relay race, where each researcher builds on the findings of the previous one.

Can you spot if we will have sequential or parallel tool calls and if we have parallel, would they really be executed in parallel?


In [ ]:
# Note, the query was modified to produce more consistent results.
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
)
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result["messages"][-1].content)

# Exercise: How would you have to change the tool definition, to allow for parallel calling of the tool?

> Note: you can omit the parallel execution with async
